In [2]:
datadir = Path("/home/jdavidson/calms21/data")
base = "/home/jdavidson/calms21/"
files = sorted(datadir.glob("*npy"))
print(f"Found {len(files)} files")

Found 6 files


In [159]:
from pathlib import Path
import os, re, json, math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, precision_recall_curve, f1_score
from tqdm import tqdm

# I/O
BASE = Path(base + "socialmapper_outputs")   # your features folder
DATADIR = Path(datadir)                      # where .npy labels live

# Task 3 label files (adjust names if yours differ)
TASK3_TRAIN = DATADIR / "calms21_task3_train.npy"
TASK3_TEST  = DATADIR / "calms21_task3_test.npy"

# Behaviors to learn (7 new ones) 
TASK3_BEHAVIORS = ['approach', 'disengaged', 'groom', 'intromission', 'mount_attempt', 'sniff_face', 'whiterearing']

# temporal-stack params (must match what you used to build X_train/X_test before)
HALF = 50
SKIP = 2
ADD_POOL = True
POOL_STATS = ("mean","std")
SIGMA_STACK = 25
SIGMA_POOL  = 30

# parallelism
N_JOBS = 10

# split
TEST_SIZE = 0.2
RANDOM_STATE = 42

# XGBoost (GPU if available)
USE_GPU = True
XGB_PARAMS = dict(
    max_depth=8,
    learning_rate=0.05,
    n_estimators=1500,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=1.0,
    objective="binary:logistic",
    tree_method=("gpu_hist" if USE_GPU else "hist"),
    eval_metric="aucpr",
    n_jobs=0,
)

# ---- expected helpers from your current codebase ----
# - to_raw_seq(safe_seq)
# - load_gt_labels_from_many(...)  <-- we won't use; Task3 has its own label dicts
# - _load_concat_wavelet(BASE, safe_seq, persp, allow_pair_only=False)
# - temporal_stack_features(X, half_window, skip, add_pool, pool_stats, apply_to, sigma_stack, sigma_pool)
# - sliding_stats_block (if you decide not to use temporal stacking)

# 2) Load Task-3 labels

In [160]:
def _load_task3_dict(p: Path):
    d = np.load(p, allow_pickle=True).item()
    return d

def load_task3_labels(train_path: Path, test_path: Path):
    dtr = _load_task3_dict(train_path) if train_path.exists() else {}
    dte = _load_task3_dict(test_path)  if test_path.exists()  else {}
    return dtr, dte

task3_train_dict, task3_test_dict = load_task3_labels(TASK3_TRAIN, TASK3_TEST)

def has_any_labels_for_sequence_task3(dsplit: dict, raw_seq_key: str) -> bool:
    for _, seqs in dsplit.items():
        if raw_seq_key in seqs:
            return True
    return False

# 3) Discover sequences & build per-sequence features


In [161]:
from pathlib import Path
import re

BASE = Path(base + "socialmapper_outputs")  # adjust if needed

# strict modern filenames only
pat = re.compile(r"wavelet_social_seq=(.*)_persp=(\d+)\.npz")

def _infer_split_from_safe(safe_seq: str) -> str:
    # filenames look like: task3-<behavior>-train-... or task3-<behavior>-test-...
    if "-train-" in safe_seq:
        return "train"
    if "-test-" in safe_seq:
        return "test"
    # fallback (shouldn't happen for Task 3)
    return "unknown"

def _list_index_for(pattern: str):
    files = sorted(BASE.glob(pattern))
    index = []
    for p_pair in files:
        m = pat.match(p_pair.name)
        if not m:
            continue
        safe_seq, persp = m.group(1), int(m.group(2))
        # require ego counterpart
        p_ego = BASE / f"wavelet_ego_seq={safe_seq}_persp={persp}.npz"
        if p_ego.exists():
            split = _infer_split_from_safe(safe_seq)
            index.append((safe_seq, persp, split))
    return sorted(set(index))

# enumerate train and test separately (you can also combine later if you want)
index_train = _list_index_for("wavelet_social_seq=task3*train*persp=*.npz")
index_test  = _list_index_for("wavelet_social_seq=task3*test*persp=*.npz")

print(f"Found {len(index_train)} TRAIN (seq,persp) with pair+ego.")
print(f"Found {len(index_test)}  TEST  (seq,persp) with pair+ego.")

Found 34 TRAIN (seq,persp) with pair+ego.
Found 218  TEST  (seq,persp) with pair+ego.


# 4) Build Task-3 dataset (temporal stack + labels)


In [162]:
import numpy as np
from pathlib import Path

# ---------- strict helpers (modern names only) ----------

def to_raw_seq(safe_seq: str) -> str:
    """CalMS21 key from 'safe' filename: replace '-' with '/'. 
       Example: task3-sniff_face-train-mouse023_task3_sniff_face -> task3/sniff_face/train/mouse023_task3_sniff_face
    """
    return safe_seq.replace("-", "/")

def _ensure_TF(arr: np.ndarray) -> np.ndarray:
    """Ensure (T,F): if rows<<cols it's likely (F,T) -> transpose."""
    if arr.ndim != 2:
        raise ValueError(f"Expected 2D array, got shape {arr.shape}")
    r, c = arr.shape
    return arr.T if r <= c else arr

def _find_pair_wavelet_file(in_dir: Path, safe_seq: str, persp: int) -> Path:
    p = in_dir / f"wavelet_social_seq={safe_seq}_persp={persp}.npz"
    if not p.exists():
        raise FileNotFoundError(f"Missing pair wavelet: {p.name}")
    return p

def _find_ego_wavelet_file(in_dir: Path, safe_seq: str, persp: int) -> Path:
    p = in_dir / f"wavelet_ego_seq={safe_seq}_persp={persp}.npz"
    if not p.exists():
        raise FileNotFoundError(f"Missing ego wavelet: {p.name}")
    return p

def _load_concat_wavelet(in_dir: Path, safe_seq: str, persp: int) -> np.ndarray:
    """Load pair + ego 'spectrogram' and hstack (truncate to min T). Returns float32 (T, F_pair+F_ego)."""
    pair_path = _find_pair_wavelet_file(in_dir, safe_seq, persp)
    ego_path  = _find_ego_wavelet_file(in_dir,  safe_seq, persp)
    with np.load(pair_path) as P:
        Xp = _ensure_TF(np.asarray(P["spectrogram"]))
    with np.load(ego_path) as E:
        Xe = _ensure_TF(np.asarray(E["spectrogram"]))
    T = min(Xp.shape[0], Xe.shape[0])
    return np.hstack([Xp[:T], Xe[:T]]).astype(np.float32, copy=False)

# ---------- label dicts + robust accessor for Task 3 ----------

def _load_task3_dict(p: Path) -> dict:
    return np.load(p, allow_pickle=True).item()

# point to your Task-3 npy files
TASK3_TRAIN = Path(datadir) / "calms21_task3_train.npy"
TASK3_TEST  = Path(datadir) / "calms21_task3_test.npy"

task3_train_dict = _load_task3_dict(TASK3_TRAIN) if TASK3_TRAIN.exists() else {}
task3_test_dict  = _load_task3_dict(TASK3_TEST)  if TASK3_TEST.exists()  else {}

def _get_task3_seq_record(dsplit: dict, raw_seq_key: str):
    """Find the record payload for a sequence in a split dict."""
    for _, seqs in dsplit.items():
        if raw_seq_key in seqs:
            return seqs[raw_seq_key]
    return None

def get_task3_seq_labels(dsplit: dict, raw_seq_key: str, behavior: str) -> np.ndarray | None:
    """
    Return (T,) 0/1 annotation vector for a given behavior and sequence.
    Expects structure:
        dsplit[behavior][raw_seq_key]['annotations'] -> array
    """
    try:
        return np.asarray(dsplit[behavior][raw_seq_key]["annotations"]).astype(int)
    except KeyError:
        return None
    except Exception as e:
        print(f"[WARN] get_task3_seq_labels failed for {behavior} / {raw_seq_key}: {e}")
        return None

# ---------- your temporal stacking (use the version you settled on) ----------


def temporal_stack_features(
    X: np.ndarray,
    half_window: int = 16,
    skip: int = 1,
    add_pool: bool = True,
    pool_stats=("mean", "std"),
    # --- Gaussian kernel controls ---
    apply_to=("scale", "pool"),     # any of {"scale", "pool"}; use () or None for none
    sigma_stack: float = None,      # Gaussian sigma (frames) for the stacked (skip-sampled) offsets
    sigma_pool: float = None        # Gaussian sigma (frames) for the dense pooling window
):
    """
    X: (T, F) per-frame features (e.g., wavelet over PCA for social pairs)
    half_window: number of frames before/after the center
    skip: take every 'skip' frame within the *stacked* window
    add_pool: append pooled features over the full dense window
    pool_stats: which pooled stats to append ("mean","std","max","min")
    apply_to: tuple indicating where to apply Gaussian weights:
              - "scale": multiply stacked time slices by Gaussian weights before flattening
              - "pool" : use Gaussian *weighted* pooling for mean/std (max/min remain unweighted)
    sigma_stack: std dev (in frames) for the stacked offsets weighting
    sigma_pool:  std dev (in frames) for the dense pooling weighting

    Returns:
      X_stack: (N_kept,  F*(num_steps)  [+ pooled dims])
      keep_idx: (N_kept,) center indices in the original time axis
    """
    if apply_to is None:
        apply_to = tuple()
    if isinstance(apply_to, str):
        apply_to = (apply_to,)

    T, F = X.shape

    # --- Offsets for the *stacked* (skip-sampled) window ---
    offsets = np.arange(-half_window, half_window + 1, skip, dtype=int)   # e.g., [-50,-45,...,0,...,45,50]
    half_eff = offsets[-1]                                                # effective half-span considering skip

    # centers we can keep without crossing sequence boundaries
    start = max(half_window, half_eff)
    end   = min(T - half_window, T - half_eff)
    if end <= start:
        return np.empty((0, F), dtype=X.dtype), np.array([], dtype=int)

    centers = np.arange(start, end, dtype=int)           # center indices (N_kept,)
    num_steps = len(offsets)

    # --- Gaussian weights for the *stacked* offsets (for "scale") ---
    if "scale" in apply_to:
        w_stack = _gaussian_weights(offsets, sigma_stack)   # shape (num_steps,)
    else:
        w_stack = np.ones(num_steps, dtype=np.float32)

    # --- Build stacked matrix, optionally Gaussian-scaled over time ---
    X_stack = np.empty((len(centers), F * num_steps), dtype=X.dtype)
    for i, t in enumerate(centers):
        window = X[t + offsets, :]  # (num_steps, F)
        if "scale" in apply_to:
            window = (window * w_stack[:, None])            # apply temporal weights to each slice
        X_stack[i] = window.reshape(-1)

    # --- Optional pooled features over the *dense* window: [-half_window, +half_window] with step 1 ---
    if add_pool:
        dense_offsets = np.arange(-half_window, half_window + 1, dtype=int)  # every frame in the window
        if "pool" in apply_to:
            w_pool = _gaussian_weights(dense_offsets, sigma_pool)             # (2*half_window+1,)
        else:
            w_pool = None  # means "flat" pooling

        pooled = []
        for t in centers:
            W = X[t - half_window : t + half_window + 1, :]  # (Twin_dense, F)
            stats_parts = []
            for stat in pool_stats:
                if stat == "mean":
                    if w_pool is None:
                        stats_parts.append(W.mean(axis=0))
                    else:
                        stats_parts.append(_weighted_mean(W, w_pool))
                elif stat == "std":
                    if w_pool is None:
                        stats_parts.append(W.std(axis=0))
                    else:
                        stats_parts.append(_weighted_std(W, w_pool))
                elif stat == "max":
                    stats_parts.append(W.max(axis=0))   # max/min are not naturally "weighted"
                elif stat == "min":
                    stats_parts.append(W.min(axis=0))
            if stats_parts:
                stats_vec = np.concatenate(stats_parts, axis=0)
                pooled.append(stats_vec)

        if pooled:  # if we actually added any stats
            pooled = np.vstack(pooled)  # (N_kept, F * len(pool_stats))
            X_stack = np.hstack([X_stack, pooled])

    return X_stack, centers
def align_labels(y, centers):
    """Pick the center-frame label for each stacked sample."""
    return y[centers]

def align_labels(y: np.ndarray, centers: np.ndarray) -> np.ndarray:
    return y[centers]

# ---------- block builder that HONORS the split inferred from filename ----------

def _infer_split_from_safe(safe_seq: str) -> str:
    if "-train-" in safe_seq: return "train"
    if "-test-"  in safe_seq: return "test"
    return "unknown"

def _one_block_task3(safe_seq: str, persp: int, behaviors: list[str]):
    raw_seq = to_raw_seq(safe_seq)
    split = _infer_split_from_safe(safe_seq)
    dsplit = task3_train_dict if split == "train" else task3_test_dict if split == "test" else None
    if dsplit is None:
        return None

    # if sequence is absent from that split's labels, skip early
    if _get_task3_seq_record(dsplit, raw_seq) is None:
        return None

    # features
    X = _load_concat_wavelet(BASE, safe_seq, persp)  # (T, F_pair+F_ego)

    # temporal stack
    Xw, centers = temporal_stack_features(
        X, half_window=HALF, skip=SKIP, add_pool=ADD_POOL, pool_stats=POOL_STATS,
        apply_to=("scale", "pool"), sigma_stack=SIGMA_STACK, sigma_pool=SIGMA_POOL
    )
    if Xw.shape[0] == 0:
        return None

    # per-behavior labels (from the *correct* split dict)
    y_per_b = {}
    T = X.shape[0]
    for b in behaviors:
        y_seq = get_task3_seq_labels(dsplit, raw_seq, b)
        if y_seq is None:
            continue
        n = min(T, y_seq.shape[0])
        y_per_b[b] = y_seq[:n][centers].astype(int)

    if not y_per_b:
        return None

    g_block = np.full(Xw.shape[0], raw_seq, dtype=object)
    return dict(X=Xw, y_per_b=y_per_b, g=g_block, seq=raw_seq, persp=persp, split=split)

In [163]:
from joblib import Parallel, delayed
from tqdm import tqdm

# Build TRAIN blocks
blocks_train = Parallel(n_jobs=N_JOBS, prefer="processes")(
    delayed(_one_block_task3)(safe_seq, persp, TASK3_BEHAVIORS)
    for (safe_seq, persp, split) in tqdm(index_train, desc="Task3 TRAIN: load+stack")
)
blocks_train = [b for b in blocks_train if b is not None]
print(f"Task3 TRAIN blocks: {len(blocks_train)}")

# Build TEST blocks
blocks_test = Parallel(n_jobs=N_JOBS, prefer="processes")(
    delayed(_one_block_task3)(safe_seq, persp, TASK3_BEHAVIORS)
    for (safe_seq, persp, split) in tqdm(index_test, desc="Task3 TEST: load+stack")
)
blocks_test = [b for b in blocks_test if b is not None]
print(f"Task3 TEST blocks: {len(blocks_test)}")





Task3 TRAIN: load+stack:   0%|                                                                          | 0/34 [00:00<?, ?it/s]



Task3 TRAIN: load+stack:  59%|██████████████████████████████████████▏                          | 20/34 [00:01<00:01, 12.25it/s]



Task3 TRAIN: load+stack: 100%|█████████████████████████████████████████████████████████████████| 34/34 [00:09<00:00,  3.50it/s]


Task3 TRAIN blocks: 34






Task3 TEST: load+stack:   0%|                                                                          | 0/218 [00:00<?, ?it/s]



Task3 TEST: load+stack:   9%|█████▉                                                           | 20/218 [00:01<00:15, 12.67it/s]/home/jdavidson/miniforge3/envs/ecodylic/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(




Task3 TEST: load+stack:  14%|████████▉                                                        | 30/218 [00:07<00:52,  3.57it/s]



Task3 TEST: load+stack:  18%|███████████▉                                                     | 40/218 [00:11<00:59,  3.00it/s]



Task3 TEST: load+stack:  23%|██████████████▉                                                  | 50/218 [00:14<00:55,  3.04it/s]



Task3 TEST: load+stack:  28%|█████████████████▉                

Task3 TEST blocks: 218


# 5 assemble per-behavior datasets 

In [164]:
import numpy as np

def _concat_blocks_for_behavior(blocks, behavior: str):
    Xs, ys, gs = [], [], []
    for b in blocks:
        if behavior in b["y_per_b"]:
            Xs.append(b["X"])
            ys.append(b["y_per_b"][behavior])
            gs.append(b["g"])
    if not Xs:
        return None, None, None
    return (np.vstack(Xs).astype(np.float32, copy=False),
            np.concatenate(ys).astype(np.int32, copy=False),
            np.concatenate(gs))

# example usage later:
# Xtr_b, ytr_b, gtr_b = _concat_blocks_for_behavior(blocks_train, "sniff_face")
# Xte_b, yte_b, gte_b = _concat_blocks_for_behavior(blocks_test,  "sniff_face")

# 6  train XGBoost per behavior (with class-imbalance handling + val split)

In [165]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import average_precision_score, f1_score, precision_recall_curve
import xgboost as xgb
from pathlib import Path
import json, os

OUT_DIR = (BASE / "task3_xgb_models")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def _train_val_split(X, y, test_size=0.2, random_state=42):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    (tr_idx, va_idx), = sss.split(X, y)
    return X[tr_idx], y[tr_idx], X[va_idx], y[va_idx]

def _scale_pos_weight(y):
    # (neg / pos) — guard rare class
    pos = np.count_nonzero(y == 1)
    neg = y.size - pos
    return float(neg / max(1, pos))

def train_xgb_for_behavior(behavior: str, Xtr, ytr, Xte, yte, *,
                           use_gpu=True, random_state=42):
    # split TRAIN into (train, val) for early stopping + threshold tuning
    X_tr, y_tr, X_va, y_va = _train_val_split(Xtr, ytr, test_size=0.2, random_state=random_state)

    params = dict(
        objective="binary:logistic",
        eval_metric="aucpr",
        tree_method="hist",
        device="cuda", # GPU acceleration
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.7,
        reg_lambda=1.0,
        reg_alpha=0.0,
        random_state=random_state,
        scale_pos_weight=_scale_pos_weight(y_tr),
        n_estimators=4000,  # we will early-stop
    )

    clf = xgb.XGBClassifier(**params)
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        verbose=False
    )

    # threshold tuning on **validation** (maximize F1 or Youden’s J on PR curve)
    va_proba = clf.predict_proba(X_va)[:, 1]
    precision, recall, thresh = precision_recall_curve(y_va, va_proba)
    f1s = np.where((precision + recall) > 0, 2 * precision * recall / (precision + recall), 0.0)
    best_i = int(np.nanargmax(f1s))
    best_thr = float(thresh[max(0, best_i - 1)]) if best_i > 0 else 0.5  # PR returns len(thresh)=len(precision)-1

    # test evaluation
    te_proba = clf.predict_proba(Xte)[:, 1]
    y_pred = (te_proba >= best_thr).astype(np.int32)
    ap = average_precision_score(yte, te_proba)
    f1 = f1_score(yte, y_pred, zero_division=0)

    # save
    beh_dir = OUT_DIR / behavior
    beh_dir.mkdir(exist_ok=True, parents=True)
    model_path = beh_dir / "xgb.json"
    clf.get_booster().save_model(model_path.as_posix())
    with open(beh_dir / "metrics.json", "w") as f:
        json.dump({
            "behavior": behavior,
            "best_iteration": int(clf.best_iteration) if hasattr(clf, "best_iteration") else None,
            "val_best_threshold": best_thr,
            "test_AP": ap,
            "test_F1": f1,
            "class_counts_train": {
                "pos": int(np.sum(y_tr==1)),
                "neg": int(np.sum(y_tr==0)),
                "scale_pos_weight": params["scale_pos_weight"],
            }
        }, f, indent=2)

    return dict(model=clf, thr=best_thr, AP=ap, F1=f1, path=model_path)

# 7 run the loop over all Task-3 behaviors and report

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

results_task3 = {}

for behavior in TASK3_BEHAVIORS:
    Xtr_b, ytr_b, _ = _concat_blocks_for_behavior(blocks_train, behavior)
    Xte_b, yte_b, _ = _concat_blocks_for_behavior(blocks_test,  behavior)

    if Xtr_b is None or Xte_b is None:
        print(f"[WARN] {behavior}: missing data; skipping.")
        continue

    print(f"\n=== Training XGB for {behavior} ===")
    print(f"  train: X={Xtr_b.shape}, pos={np.sum(ytr_b==1)}, neg={np.sum(ytr_b==0)}")
    print(f"  test : X={Xte_b.shape}, pos={np.sum(yte_b==1)}, neg={np.sum(yte_b==0)}")

    out = train_xgb_for_behavior(behavior, Xtr_b, ytr_b, Xte_b, yte_b, use_gpu=True)

    # detailed test report (at tuned threshold)
    yhat = (out["model"].predict_proba(Xte_b)[:,1] >= out["thr"]).astype(np.int32)
    print(f"--- {behavior} test report ---")
    print(classification_report(yte_b, yhat, digits=3, zero_division=0))
    print("Confusion matrix:\n", confusion_matrix(yte_b, yhat))
    print(f"AP={out['AP']:.4f}  F1={out['F1']:.4f}  thr={out['thr']:.3f}  -> {out['path'].name}")

    results_task3[behavior] = dict(AP=out["AP"], F1=out["F1"], thr=out["thr"])

# quick summary
print("\n=== Task 3 summary (TEST) ===")
for b, r in results_task3.items():
    print(f"{b:15s}  AP={r['AP']:.4f}  F1={r['F1']:.4f}  thr={r['thr']:.3f}")
print("Macro averages:",
      "AP=", np.mean([r["AP"] for r in results_task3.values()]).round(4),
      "F1=", np.mean([r["F1"] for r in results_task3.values()]).round(4))


=== Training XGB for approach ===
  train: X=(40648, 4770), pos=1328, neg=39320
  test : X=(247936, 4770), pos=7572, neg=240364
--- approach test report ---
              precision    recall  f1-score   support

           0      0.971     0.990     0.980    240364
           1      0.149     0.058     0.083      7572

    accuracy                          0.961    247936
   macro avg      0.560     0.524     0.532    247936
weighted avg      0.946     0.961     0.953    247936

Confusion matrix:
 [[237864   2500]
 [  7134    438]]
AP=0.0723  F1=0.0833  thr=0.024  -> xgb.json

=== Training XGB for disengaged ===
  train: X=(71102, 4770), pos=1048, neg=70054
  test : X=(37976, 4770), pos=456, neg=37520
